# **IMPORT LIBRERIE**

In [1]:
import carla, time, random

from Adas import Forward_collision_warning_mqtt

# **CARLA**

In [2]:
# Connessione
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)

# Instanza mondo
world = client.get_world()

# Caricamento mappa
if not world.get_map().name == 'Carla/Maps/Town03':
    world = client.load_world('Town03')

# **METODI PER LA CREAZIONE DI ATTORI**

In [3]:
def spawn_vehicle_by_spawnPoint(vehicle_index=0, spawn_index=0, pattern='vehicle.*'):
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter(pattern)[vehicle_index]
    spawn_point = world.get_map().get_spawn_points()[spawn_index]
    vehicle = world.spawn_actor(vehicle_bp, spawn_point)
    return vehicle

In [4]:
def spawn_vehicle_by_location(vehicle_index=0, location=carla.Location(), rotation = carla.Rotation()):
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter('vehicle.*')[vehicle_index]
    spawn_point = carla.Transform()
    spawn_point.location = location
    spawn_point.rotation = rotation
    vehicle = world.spawn_actor(vehicle_bp, spawn_point)
    return vehicle

In [5]:
def spawn_walker():
    walker_bp = random.choice(world.get_blueprint_library().filter("walker.pedestrian.*"))

    start_location = carla.Location(x=70, y=25, z=1)

    trans = carla.Transform()
    trans.location = start_location

    return world.spawn_actor(walker_bp, trans)

# **FUNZIONI DI SUPPORTO**

In [6]:
# Comandi guida veicolo
def move_forward(vehicle):
    global throttle
    control = carla.VehicleControl()
    control.throttle = throttle
    vehicle.apply_control(control)

def stop_vehicle(vehicle):
    control = carla.VehicleControl()
    control.throttle = 0.0
    control.brake = 1
    vehicle.apply_control(control)

In [7]:
# Simulazione sensore umidità dell'asflato
def get_asphalt_friction_coefficient():
    world = client.get_world()
    weather = world.get_weather()
    weather.precipitation_deposits
    weather.wetness
    return 0.8

In [8]:
# Uscita dalla simulazione
def stop_simulation():
    global run
    run = False
    print("stop")
    
    

In [9]:
def get_vehicle_velocity(vehicle):
    velocity_vector = vehicle.get_velocity()
    velocity = (velocity_vector.x**2 + velocity_vector.y**2 + velocity_vector.z**2)**0.5
    if vehicle.get_control().reverse:
        velocity = -velocity
    return velocity

# **SIMULAZIONE**

In [ ]:
try:
    vehicle = spawn_vehicle_by_location(0, carla.Location(x=140, y=-5.8, z=1), carla.Rotation(yaw = 180))
    #vehicle = spawn_vehicle_by_spawnPoint()
    #walker = spawn_walker()

    run = True
    throttle = 0.7

    adas = Forward_collision_warning_mqtt(
        world = world,
        attached_vehicle = vehicle,
        get_asphalt_friction_coefficient = lambda : get_asphalt_friction_coefficient(),
        action_listener = lambda : stop_simulation()
    )

    vehicle.set_autopilot(True)

    while run:
        #move_forward(vehicle)
        time.sleep(0.05)
    
    stop_vehicle(vehicle)
    time.sleep(10)

finally:
    adas.destroy()
    vehicle.destroy()
    #walker.destroy()
    